In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from commando import ComManDo
from commando.evaluation import test_partial
import numpy as np
from sklearn.metrics import confusion_matrix
import torch

2022-01-18 18:44:57.019627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 18:44:57.019660: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data1 = np.loadtxt("../data/UnionCom/simu1/domain1.txt")
data2 = np.loadtxt("../data/UnionCom/simu1/domain2.txt")
num_aligned = int(.8 * len(data2))
data2, data2_withheld = data2[:num_aligned], data2[num_aligned:]

# Predict Missing Modality

In [4]:
# Align data
cm = ComManDo(aligned_idx=2*(range(num_aligned),))
cm_data = cm.fit_transform(dataset=[data1, data2])

use random seed: 666
Shape of Raw data
Dataset 0: (300, 1000)
Dataset 1: (240, 500)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[100/2000] err:0.9915 alpha:0.3969
epoch:[200/2000] err:0.8619 alpha:0.3508
epoch:[300/2000] err:0.8517 alpha:0.3583
epoch:[400/2000] err:0.8388 alpha:0.3670
epoch:[500/2000] err:0.8213 alpha:0.3764
epoch:[600/2000] err:0.7994 alpha:0.3864
epoch:[700/2000] err:0.7735 alpha:0.3970
epoch:[800/2000] err:0.7417 alpha:0.4083
epoch:[900/2000] err:0.7004 alpha:0.4208
epoch:[1000/2000] err:0.6481 alpha:0.4348
epoch:[1100/2000] err:0.5905 alpha:0.4494
epoch:[1200/2000] err:0.5388 alpha:0.4628
epoch:[1300/2000] err:0.4998 alpha:0.4736
epoch:[1400/2000] err:0.4721 alpha:0.4817
epoch:[1500/2000] err:0.4513 alpha:0.4879
epoch:[1600/2000] err:0.4347 alpha:0.4930
epoch:[1700/2000] err:0.4209 alpha:0.4973
epoch:[1800/2000] err:0.4089 alpha:0.5012
epoch:[1900/2000] err:0.3979 alpha:0.5047
epoch:[2000/2000] err:0.3875 

In [5]:
# Compile prediction and true
predicted = cm.model.decoders[1](torch.tensor(cm_data[0][num_aligned:])).detach().cpu().numpy()
predicted = predicted.flatten() > .5
actual = (data2_withheld - np.min(data2_withheld)) / (np.max(data2_withheld) - np.min(data2_withheld))
actual = actual.flatten() > .5

In [6]:
# Produce confusion matrix
# tn fp
# fn tp
confusion_matrix(actual, predicted, labels=(False, True), normalize='true')

array([[0.36637635, 0.63362365],
       [0.0158591 , 0.9841409 ]])